### Setting up

In [1]:
# libraries
import numpy as np
import glob
import os
import shutil
import time
from IPython.core.debugger import Tracer

In [2]:
# pytorch modules
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [3]:
# seed
np.random.seed(7)

In [4]:
# intermediate folder
intermediate_dir = "../src/intermediate/"
if not os.path.isdir(intermediate_dir):
    os.mkdir(intermediate_dir)

In [5]:
# global information
traindir = os.path.join(intermediate_dir, "train")
valdir = os.path.join(intermediate_dir, "val")
valnum = 500 # number of validate image
num_workers = 5
epochs = 90
batch_size = 16
learning_rate = 1e-4
momentum = 0.9
weight_decay = 1e-4
print_freq = 10
pretrained = True

### Train/val split

In [6]:
# create intermediate folder, copy train data, and split
if not os.path.isdir(traindir):
    shutil.copytree("../data/train/", traindir)

if not os.path.isdir(valdir):
    g = glob.glob(traindir + "*/*.jpg")
    shuf = np.random.permutation(g)
    for i in range(valnum):
        os.renames(shuf[i], shuf[i].replace("train", "val"))

### Basic model

In [7]:
# create model
model = models.resnet50(pretrained=pretrained)

for param in model.parameters():
    param.requires_grad = False
# replace the last fully-connected layer
bn_expansion = 4
model.fc = nn.Linear(512 * bn_expansion, 8)
model = torch.nn.DataParallel(model).cuda()

In [8]:
# set this flag to get 2x speed improvement
# according to PyTorch Slack #beginner channel
cudnn.benchmark = True

In [9]:
# Data loading code
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(traindir, transforms.Compose([
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

In [10]:
k = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [12]:
k.classes

['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [16]:
train_loader.dataset.class_to_idx

{'.DS_Store': 0,
 'ALB': 1,
 'BET': 2,
 'DOL': 3,
 'LAG': 4,
 'NoF': 5,
 'OTHER': 6,
 'SHARK': 7,
 'YFT': 8}

In [10]:
# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.Adam(model.module.fc.parameters(), learning_rate,
#                             momentum=momentum,
                             weight_decay=weight_decay)

In [11]:
def adjust_learning_rate(optimizer, epoch):
    """Set the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learning_rate * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [12]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(images)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        Tracer()()
        loss = criterion(output, target_var)
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        top1.update(prec1[0], images.size(0))
        top5.update(prec5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))

In [13]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (images, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(images, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        Tracer()()
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        top1.update(prec1[0], images.size(0))
        top5.update(prec5[0], images.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top1.avg

In [14]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    filename = os.path.join(intermediate_dir, filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(intermediate_dir,
                                               'model_best.pth.tar'))

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [16]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learning_rate * (0.1 ** (epoch // 30))
    for param_group in optimizer.state_dict()['param_groups']:
        param_group['lr'] = lr

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
best_prec1 = 0

for epoch in range(epochs):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    prec1 = validate(val_loader, model, criterion)

    # remember best prec@1 and save checkpoint
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
    }, is_best)

> <ipython-input-12-f15194edb1eb>(23)train()
     21         output = model(input_var)
     22         Tracer()()
---> 23         loss = criterion(output, target_var)
     24 
     25         # measure accuracy and record loss

ipdb> target_var
Variable containing:
 1
 5
 5
 2
 6
 1
 1
 8
 1
 8
 8
 8
 6
 1
 1
 6
[torch.cuda.LongTensor of size 16 (GPU 0)]

